In [1]:
import pandas as pd
df = pd.read_csv("perm_importance_mets.csv")
df

,coralonly_rf,coralonly_gb,all_rfgb
0,x22863_768_59077_10_971,x22863_768_59077_10_971,x19344_800_60417_9_097
1,x3156_482_36064_2_937,x3156_482_36064_2_937,x25770_796_62218_12_134
2,x20449_766_57519_9_667,x19279_643_51751_9_054,x19279_643_51751_9_054
3,x23789_794_60652_11_52,x22902_794_60499_10_997,x6124_510_39212_3_959
4,x24987_796_62223_11_912,x26021_752_55957_12_218,x1935_553_33175_1_992
...,...,...,...
62,NaN,x27604_822_63483_12_635,NaN
63,NaN,x11971_465_3365_6_009,NaN
64,NaN,x18859_696_5408_8_846,NaN
65,NaN,x11174_708_4958_5_756,NaN
